In [5]:
import pandas as pd
import plotly.express as px

from sklearn.feature_selection import VarianceThreshold
from sklearn.pipeline import Pipeline

from utils.data_preparation import *
from utils.data_exploration import *

from rdkit import Chem


task = 'CYP2C19'

data = data_preprocessing(task)
data = data.drop(["MACCS_FP", "ATOMPAIR_FP"], axis=1)
data_nan = extract_null(data)
data = data.drop(data_nan.index)
datasets = dataset_split(data.drop(["Drug", "Drug_ID"], axis=1))

continuous_data_train = datasets["train"].select_dtypes(include='float64').join(datasets["train"]["Y"])
discrete_data_train = datasets["train"].select_dtypes(include='int64')


datasets["train"]


Dataset already exists, returning data/cyp2c19/raw_dataset.csv.


,Y,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,...,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,Morgan_FP
8939,0,11.537246,-0.014776,11.537246,0.014776,0.494847,283.378,270.274,283.044919,96,...,0,0,0,0,0,0,1,1,0,0000000000000110000000000000000010000000010000...
11670,1,11.738975,-0.342543,11.738975,0.236402,0.826261,334.828,319.708,334.054276,114,...,0,0,0,0,0,0,0,0,0,0100000000000000000100000000000000000000010000...
4326,0,11.743380,-0.953103,11.743380,0.254207,0.720956,228.244,212.116,228.099774,90,...,0,0,0,0,0,0,0,0,0,0000000000000100000000000000000000000000000000...
11437,1,11.183975,-0.492198,11.183975,0.079030,0.267923,403.423,390.319,403.073910,142,...,0,0,0,0,0,0,0,0,0,0000000000000000000000000000000010000000000000...
8146,0,12.580185,-0.122551,12.580185,0.013915,0.800347,354.406,332.230,354.157957,136,...,0,0,0,0,0,0,0,0,0,0000000000000000000000000000100000000000000000...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3450,1,12.980420,-3.731463,12.980420,0.074466,0.707613,475.014,447.798,474.149239,170,...,1,0,0,0,0,0,0,0,0,0000000000100000000000000100000000000000000000...
4615,0,10.762461,-1.033866,10.762461,0.047665,0.679386,209.201,198.113,209.068808,80,...,0,0,0,0,0,0,0,0,0,0000000000000000000000000000000000000000100000...
7387,1,13.311038,-0.319724,13.311038,0.133525,0.672273,424.505,396.281,424.222289,164,...,0,0,0,1,0,0,0,0,0,0110000000000000000000000000000000000000000000...
205,0,13.421800,-0.126117,13.421800,0.126117,0.391930,392.463,368.271,392.196074,150,...,0,0,0,0,0,0,0,0,0,0100000000000000000000000000000000000100000000...


In [25]:
from sklearn.decomposition import PCA

pca = PCA(n_components=5)
pca.fit(continuous_data_train.drop("Y", axis=1))

PCA(n_components=5)

In [27]:
pca.get_covariance()

array([[-1.40211880e+47, -2.96932474e+43,  3.42603642e+43, ...,
        -6.94998243e+41, -5.54737143e+43, -2.33443972e+44],
       [-2.96932474e+43, -1.40211671e+47, -3.23241162e+43, ...,
         4.72360982e+42,  4.17335425e+43,  3.14968513e+44],
       [ 3.42603642e+43, -3.23241162e+43, -1.40209338e+47, ...,
        -1.78307279e+42, -5.46192489e+43, -2.62152101e+44],
       ...,
       [-6.94998243e+41,  4.72360982e+42, -1.78307279e+42, ...,
        -1.40242368e+47, -4.00905514e+42,  7.03005237e+43],
       [-5.54737143e+43,  4.17335425e+43, -5.46192489e+43, ...,
        -4.00905514e+42, -1.40113899e+47,  5.40857695e+44],
       [-2.33443972e+44,  3.14968513e+44, -2.62152101e+44, ...,
         7.03005237e+43,  5.40857695e+44, -1.34304522e+47]])

In [28]:
pca.transform(continuous_data_train.drop("Y", axis=1).values)

array([[-6.06621065e+29, -3.26112877e+02,  2.15416252e+01,
         1.73139339e+01,  2.14273542e+01],
       [-6.06621065e+29, -1.74132454e+02,  3.93335032e+01,
        -3.56461758e+01,  5.56469626e+01],
       [-6.06621065e+29, -5.76354994e+02,  4.63131129e+01,
         4.36283910e+01, -1.77048064e+01],
       ...,
       [-6.06621065e+29,  3.53822420e+02, -1.20985494e+01,
         1.89704546e+01, -7.61790505e+01],
       [-6.06621065e+29,  4.68045046e+02, -1.36618782e+02,
         3.96324408e+01, -4.28300983e+01],
       [-6.06621065e+29,  3.16605399e+02, -6.35717195e+01,
         1.35532553e+01, -1.67138969e+01]])